<a href="https://colab.research.google.com/github/qwerlarlgus/Object_tracking/blob/main/main5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [ ]:
def nms(dets, thresh=0.65):
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep

In [ ]:
import cv2
#from tracker import *

In [ ]:
# Create tracker object
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture("B3-Parking.avi")
#cap = cv2.VideoCapture("pklot.mp4")
# Object detection from Stable camera#object_detector = cv2.createBackgroundSubtractorKNN(his

object_detector = cv2.createBackgroundSubtractorMOG2(varThreshold = 150, detectShadows=False)

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape

    # Extract Region of interest
    roi = frame[110: 160, 60: 90] # Y, X  28 93 79 50
    #roi = frame[100: 250, 400: 600] # Y, X
    #roi = frame[10: 400, 10: 600]


    # 1. Object Detection
    mask = object_detector.apply(roi)
    #_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)  # THRESH_BINARY_INV
    # mask = cv2.adaptiveThreshold(mask,127,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,15,2)
    mask = cv2.adaptiveThreshold(mask,127,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,25,2)
    # https://opencv-python.readthedocs.io/en/latest/doc/09.imageThresholding/imageThresholding.html
    
    
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 500:
            cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(cnt)


            detections.append([x, y, w, h])

    # 2. Object Tracking
    
    #boxes_ids = tracker.update(detections)
    # NMS 
    boxes_ids = nms(tracker.update(detections))

    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.putText(roi, str(id), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

    cv2.imshow("roi", roi)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)

    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{0: (15, 25)}
{2: (15, 25)}
{2: (15, 25)}
{2: (15, 25)}
{2: (15, 25)}
{2: (15, 25)}
{2: (15, 25)}
{2: (15, 25)}
{2: (15, 25)}
{2: (15, 25)}
{2: (15, 25)}
{2: (15, 25)}
{2: (15, 25)}
{2: (1